# THESIS - Model 

# Imports 

In [9]:
# imports
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0
%load_ext line_profiler
import os
import sys
import importlib

import numpy as np
from sklearn.utils import compute_class_weight
import pandas as pd

sys.path.append('/home/ubuntu/')
os.chdir('/home/ubuntu/')
import src.model
importlib.reload(src.model)
import src.model as mdl

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0
The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [2]:

import shlex
import subprocess
import os
import sys

import numpy as np
from numpy import array
from numpy import argmax

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from scikeras.wrappers import KerasClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import PrecisionRecallDisplay
import sklearn.metrics as metrics

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.optimizers.legacy import Adam
import keras_tuner

from Bio import SeqIO
import time
import datetime

import matplotlib.pyplot as plt
import pylab as pl
import seaborn as sns

import importlib
import src
importlib.reload(src.model)
import src.model as mdl

sys.path.append('/home/ubuntu/')
os.chdir('/home/ubuntu/')

In [3]:
# all paths
datapath = '/mnt/data/'
resultspath = '/mnt/data/results/'
predictionspath = '/mnt/data/results/all_predictions/'
plotspath = '/mnt/data/results/plots/'

# 1. Preparation of the dataset 
- One hot encoding the sequences 
- Dataset Split 

# One hot encoding
def fasta_to_onehotencode(seq):
    # values = list(seq)
    # values = np.array(values)
    base2int = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    # label = int(line.strip()[1])

    sequence = seq
    # Encode sequence bases as integers, i.e. A as 0, C as 1, etc.
    sequence_int = [base2int.get(base, 9999) for base in sequence]

    sequence_onehot = tf.one_hot(sequence_int, depth=4)

    return sequence_onehot

def fastatoarray(fasta_sequences):
    seq_array = np.zeros((1, 400, 4))
    for fasta in fasta_sequences:
        name, sequence = fasta.id, str(fasta.seq)
        new_sequence = fasta_to_onehotencode(sequence)  # onehotencode(sequence)
        new_sequence = np.expand_dims(new_sequence, axis=0)
        seq_array = np.vstack((seq_array, new_sequence))
    seq_array = np.delete(seq_array, 0, 0)  # to remove the first array of zeros
    return seq_array

_starting_time = time.time()
# change a bit this function or cite


def tic():
    global _starting_time
    _starting_time = time.time()


def tac():
    t_sec = round(time.time() - _starting_time)
    (t_min, t_sec) = divmod(t_sec, 60)
    (t_hour, t_min) = divmod(t_min, 60)
    print('Time passed: {}hour:{}min:{}sec'.format(t_hour, t_min, t_sec))

def prepare_dataset(path):
    current_folder = folder
    directory2 = os.listdir(f'{path}/{folder}')
    seq_list_positive = np.zeros((1, 400, 4))
    seq_list_negative = np.zeros((1, 400, 4))
    seq_list_positive_test = np.zeros((1, 400, 4))
    seq_list_negative_test = np.zeros((1, 400, 4))

    for folder2 in directory2:
        directory3 = os.listdir(f'{path}{folder}/{folder2}')

        if folder2 != "fold-4":  # to be split into training and validation
            seq_list_positive_file = np.zeros((1, 400, 4))
            seq_list_negative_file = np.zeros((1, 400, 4))

            for file in directory3:

                if "fasta" in file and 'out' not in file:
                    fasta_sequences = SeqIO.parse(open(f'{path}{folder}/{folder2}/{file}'),
                                                  'fasta')
                    seq_array = fastatoarray(fasta_sequences)

                    if "positive" in file:
                        seq_list_positive_file = np.vstack((seq_list_positive_file,
                                                            seq_array))

                    if "negative-1" in file:  # 2 neg: 1 pos with 'negative' --> 1 neg : 1 pos 'negative-1'
                        seq_list_negative_file = np.vstack((seq_list_negative_file,
                                                            seq_array))

            seq_list_negative_file = np.delete(seq_list_negative_file, 0, 0)
            seq_list_positive_file = np.delete(seq_list_positive_file, 0, 0)

            seq_list_positive = np.vstack((seq_list_positive,
                                           seq_list_positive_file))
            seq_list_negative = np.vstack((seq_list_negative,
                                           seq_list_negative_file))

        if folder2 == "fold-4":  # this folder will be used for testing

            seq_list_positive_test_file = np.zeros((1, 400, 4))
            seq_list_negative_test_file = np.zeros((1, 400, 4))

            for file in directory3:

                if "fasta" in file and 'out' not in file:
                    fasta_sequences = SeqIO.parse(open(f'{path}/{folder}/{folder2}/{file}'),
                                                  'fasta')
                    seq_array = fastatoarray(fasta_sequences)

                    if "positive" in file:
                        seq_list_positive_test_file = np.vstack((seq_list_positive_test_file,
                                                                 seq_array))

                    if "negative-1" in file:
                        seq_list_negative_test_file = np.vstack((seq_list_negative_test_file,
                                                                 seq_array))

                seq_list_negative_test_file = np.delete(seq_list_negative_test_file,
                                                        0, 0)
                seq_list_positive_test_file = np.delete(seq_list_positive_test_file,
                                                        0, 0)

                seq_list_positive_test = np.vstack((seq_list_positive_test,
                                                    seq_list_positive_test_file))
                seq_list_negative_test = np.vstack((seq_list_negative_test,
                                                    seq_list_negative_test_file))

                seq_list_negative_test = np.delete(seq_list_negative_test,
                                                   0, 0)
                seq_list_positive_test = np.delete(seq_list_positive_test,
                                                   0, 0)

    seq_list_negative = np.delete(seq_list_negative,
                                  0, 0)
    seq_list_positive = np.delete(seq_list_positive,
                                  0, 0)

    # Preparation of the labels
    labels_positive = np.ones((np.shape(seq_list_positive)[0], 1))
    labels_negative = np.zeros((np.shape(seq_list_negative)[0], 1))
    labels_positive_test = np.ones((np.shape(seq_list_positive_test)[0], 1))
    labels_negative_test = np.zeros((np.shape(seq_list_negative_test)[0], 1))
    print('Shape of labels: \n-positive : ', np.shape(labels_negative),
          '\n-negative : ', np.shape(labels_negative),
          '\n-positive validation : ', np.shape(labels_positive_test),
          '\n-negative validation : ', np.shape(labels_negative_test))

    # Merging datasets
    x = np.vstack((seq_list_positive, seq_list_negative))
    x_test = np.vstack((seq_list_positive_test, seq_list_negative_test))
    y = np.vstack((labels_positive, labels_negative))
    y_test = np.vstack((labels_positive_test, labels_negative_test))

    # Splitting dataset  : test, train and validation sets
    test_size = 0.2
    x_train, x_val, y_train, y_val = train_test_split(x, y,
                                                      test_size=test_size,
                                                      shuffle=True)
    x_test, y_test = shuffle(x_test, y_test, random_state=0)

    y_train = y_train.astype("float32")  # actually useful?
    y_val = y_val.astype("float32")
    y_test = y_test.astype("float32")

    x_train = x_train.astype("float32")
    x_val = x_val.astype("float32")
    x_test = x_test.astype("float32")

    print('Shape of datasets: \n-training set : ', np.shape(x_train),
          '\n-validation set : ', np.shape(x_val),
          '\n-testing set : ', np.shape(x_test))

    # Checking class ditribution in the whole dataset and training set

    print('Label frequencies among the dataset')
    plt.hist(y)
    plt.xticks(range(2))
    plt.title('Label Frequency')
    plt.show()

    plt.hist(y_train)
    plt.xticks(range(2))
    plt.title('Label Frequency training set')
    plt.show()

    plt.hist(y_val)
    plt.xticks(range(2))
    plt.title('Label Frequency validation set')
    plt.show()
    return x_train, y_train, x_val, y_val, x_test, y_test

# 2. Standard Model  

# 2.1 Create the baseline model 
- Initial model architecture inspired by pysster (citation) 

def create_baseline():

    METRICS = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(curve='ROC', name='auroc'),
        keras.metrics.AUC(curve='PR', name='auprc')
    ]

    # Parameters 
    input_shape = (400, 4)
    filters = 30
    kernel_size = 25
    pool_size = 2
    strides = 2
    loss = 'binary_crossentropy'
    optimizer = 'adam'
    metrics = ['accuracy']

    # Create model
    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    model.add(keras.layers.Conv1D(filters=filters, kernel_size=kernel_size,
              kernel_initializer='random_normal',
              activation='relu'))
    model.add(keras.layers.MaxPooling1D(pool_size=pool_size, strides=strides,
                                        padding='valid'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Conv1D(filters=filters, kernel_size=kernel_size,
              kernel_initializer='random_normal',
              activation='relu', input_shape=input_shape ))
    model.add(keras.layers.MaxPooling1D(pool_size=pool_size, strides=strides,
                                        padding='valid'))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(1024, activation='relu'))
    model.add(keras.layers.Dropout(0.6))
    model.add(keras.layers.Dense(128, activation='relu'))
    model.add(keras.layers.Dropout(0.6))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=METRICS)

    return model

# Summary of the model
model = create_baseline()

print(model.layers)

print(model.summary())

# 2.2 Fitting and Evaluating the Model

print("Fit model on training data")


def fit_standard(x_train, y_train, x_val, y_val, folder):

    model = create_baseline()

    # Parameters
    epochs = 50
    my_callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=5),  # introducing EarlyStopping to avoid overfitting
        #tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    ]

    history = model.fit(x_train, y_train, epochs=epochs,
                        validation_data=(x_val, y_val),
                        callbacks=my_callbacks)

    # list all data in history

    # Summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # Summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # Summarize history for auroc 
    plt.plot(history.history['auroc'])
    plt.plot(history.history['val_auroc'])
    plt.plot(history.history['auprc'])
    plt.plot(history.history['val_auprc'])
    plt.title('model auroc and auprc')
    plt.ylabel('auroc and auprc')
    plt.xlabel('epoch')
    plt.legend(['auroc_train', 'auroc_test',
                'auprc_train', 'auprc_test'],
               loc='upper left')
    plt.show()

    model.save(f'{resultspath}best_models/standard_model_%s' % current_folder)

# 2.3 Testing and saving the Standard model
- Testing on the same folder on which later the architecture is going to be tuned, for comparison. 

def test_standard(model_path, x_test, y_test, folder):
    model = keras.models.load_model(model_path)

    # Testing and saving the predictions
    y_pred = model.predict(x_test)

    '''def plot_PRC(y_test, y_pred): 
        plot = PrecisionRecallDisplay.from_predictions(y_test, y_pred)
        pl= plot.ax_.set_title("Precision-Recall curve")
        return pl'''
    # Precision Recall Curve
    plt = mdl.plot_PRC(y_test, y_pred)
    plt.savefig(f'{plotspath}PRC{current_folder}_standard.png',
                bbox_inches='tight')
    plt.show()

    '''def plot_ROC(y_test, y_pred):

        false_pr, true_pr, thresholds = metrics.roc_curve(y_test, y_pred, drop_intermediate=False)

        # Generate figure

        fig = pl.figure(figsize=(14,7))
        ax = fig.add_subplot(121)

        pl.plot(false_pr, true_pr, label=" (AUC=%.2f)" % metrics.roc_auc_score(y_test, y_pred))
        ax.plot([0,1], [0,1], color="grey",label="Random Classifier",linestyle="--")
        ax.set_xlabel("False Positive Rate")
        ax.set_ylabel("True Positive Rate")
        ax.set_ylim(0,1)
        ax.set_xlim(0,1)
        ax.grid(color="#CCCCCC")
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        pl.legend()
        pl.tight_layout()
        return pl
    '''
    # Receiver Operating Characteristics 
    plt = mdl.plot_ROC(y_test, y_pred)
    plt.savefig(f'{plotspath}ROC{current_folder}_standard.png',
                bbox_inches='tight')
    plt.show()

# 3. Finding the best model via Random Search
- Identification of the optimal parameters
- Saving of the model after its evaluation

def build_model(hp):

    METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(curve='ROC', name='auroc'),
      keras.metrics.AUC(curve='PR', name='auprc')
    ]

    input_shape = (400, 4)

    # Hyperparameter search

    hp_filters = hp.Int('filters', min_value=10, max_value=60, step=10)
    hp_kernel_size = hp.Int('kernel_size', min_value=10, max_value=60, step=5)
    hp_pool_size = hp.Int('pool_size', min_value=1, max_value=10, step=1)
    hp_strides = hp.Int('strides', min_value=1, max_value=10, step=1)
    hp_learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2,
                                sampling="log")
    # hp_padding = hp.Choice('padding', ['valid','same'])
    hp_padding = 'same'
    hp_kernel_initializer = hp.Choice('initializer',
                                      ['random_normal', 'random_uniform'])
    hp_activation = hp.Choice("activation", ["relu", "tanh"])

    # Create model

    model = keras.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    model.add(keras.layers.Conv1D(filters=hp_filters, kernel_size=hp_kernel_size,
              kernel_initializer=hp_kernel_initializer,
              activation=hp_activation, input_shape=input_shape))
    model.add(keras.layers.MaxPooling1D(pool_size=hp_pool_size,
                                        strides=hp_strides, padding=hp_padding))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Conv1D(filters=hp_filters,
                                  kernel_size=hp_kernel_size,
                                  kernel_initializer=hp_kernel_initializer,
                                  activation=hp_activation,
                                  input_shape=input_shape))
    model.add(keras.layers.MaxPooling1D(pool_size=hp_pool_size,
                                        strides=hp_strides, padding=hp_padding))
    model.add(keras.layers.Dropout(0.3))
    model.add(keras.layers.Flatten())  # without parameters
    for i in range(hp.Int("num_layers", 1, 3)):
        model.add(
            layers.Dense(
                # Tune number of units separately.
                units=hp.Int(f"units_{i}", min_value=32, max_value=1024, step=32),
                activation=hp_activation
                )
        )
    if hp.Boolean("dropout"):
        model.add(layers.Dropout(rate=0.25))
    model.add(layers.Dense(10, activation=hp_activation))
    model.add(keras.layers.Dropout(0.6))
    model.add(keras.layers.Dense(1, activation='sigmoid'))

    # many models train better when reducing the learning rate gradually -- not in this case
    # lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
                                                                 # 0.001,
                                                                 # decay_steps=50*1000,
                                                                 # decay_rate=1,
                                                                  #staircase=False)

    # Compile model
    model.compile(loss='binary_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(hp_learning_rate),
                  metrics=METRICS)

    return model

def prepare_best_model(x_train, y_train, x_val, y_val, x_test, y_test):
    METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(curve='ROC', name='auroc'),
      keras.metrics.AUC(curve='PR', name='auprc')
    ]

    my_callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
        # restore best weights added later), #better to reduce to 2 for the search 
        tf.keras.callbacks.ModelCheckpoint(
            filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
        # tf.keras.callbacks.TensorBoard(log_dir='./logs'),
        ]

    model = build_model(keras_tuner.HyperParameters())
    print(model.summary())

    tuner = keras_tuner.RandomSearch(
        hypermodel=build_model,
        # objective=keras_tuner.Objective("val_auroc", direction="max"), #val_loss , val_accuracy ...
        # objective = keras_tuner.Objective("val_loss", direction="min"),
        objective="val_accuracy",
        max_trials=200,
        executions_per_trial=1,  # probably better to change to 1 and increase the max_trials
        overwrite=False,  # 'False' because I found the best combination
        directory="my_dir",
        project_name="models",
        max_consecutive_failed_trials=10,
    )

    print(tuner.search_space_summary())
    tuner.search(x_train, y_train, epochs=5, validation_data=(x_val, y_val))
    '''
    # Get the top 2 models.
    models = tuner.get_best_models(num_models=4)
    best_model = models[0]
    # Build the model.
    best_model.build(input_shape=(None, 400, 4))
    best_model.summary() #is this block relevant? '''
    

    best_hps = tuner.get_best_hyperparameters(5)
    #SAVE THESE HYPERPARAMETERS AND USE THEM FOR create_baseline_model()
    #at this point the optimal parameters have been identified, the next part of training and prediction is the same for all models 
    
    # Build the model with the best hp.
    model = build_model(best_hps[0])
    
    #saving the best model for testing and aggregated evaluation later
    model.save(f'{resultspath}best_models/best_hps_model') 


tuner.results_summary()


def training_model(x_train, y_train, x_val, y_val, x_test, y_test, folder, resultspath):
    
    METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(curve='ROC', name='auroc'),
      keras.metrics.AUC(curve='PR', name='auprc')  
    ]
    
    my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=5), #better to reduce to 2 for the search 
    tf.keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    
    # Fitting the model
    model = keras.models.load_model(f'{resultspath}best_models/best_hps_model')

    history = model.fit(x_train, y_train, epochs=200,
                        validation_data= (x_val,y_val),
                        callbacks = my_callbacks)
    
    # list all data in history --> add name of the rbp!!!!
    print(history.history.keys())
    
    plt.title('Best model evaluation for %s' %folder)
    
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy %s'  %folder)
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss %s'  %folder)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper left')
    plt.show()

    # summarize history for auroc 
    plt.plot(history.history['auroc'])
    plt.plot(history.history['val_auroc'])
    plt.plot(history.history['auprc'])
    plt.plot(history.history['val_auprc'])
    plt.title('model auroc and auprc %s'  %folder)
    plt.ylabel('auroc and auprc')
    plt.xlabel('epoch')
    plt.legend(['auroc_train', 'auroc_test','auprc_train', 'auprc_test'], loc='upper left')
    plt.show()
    
    
    model.save(f'{resultspath}best_models/best_model_base%s' %folder)

# 3.2 Testing the model 
- Testing using different methods (Precision Recall Curve, ROC Curve, Confusion Matrix)
- Saving the predictions

def testing_model(path, current_folder, x_test, y_test) : 
    #reloading the model to compute the predictions
    reconstructed_model = keras.models.load_model(path)
    y_pred = reconstructed_model.predict(x_test)
    threshold = 0.5
    y_pred_thresh = np.where(y_pred > threshold, 1,0)
    
    # Precision Recall Curve 
    plt = mdl.plot_PRC(y_test, y_pred)
    plt.savefig(f'{plotspath}PRC{current_folder}_opt.png', bbox_inches='tight')
    plt.show()
    
    # Receiver Operating Characteristics
    plt = mdl.plot_ROC(y_test, y_pred)
    plt.savefig(f'{plotspath}ROC{current_folder}_opt.png', bbox_inches='tight')
    plt.show()

    # Confusion Matrix
    plt= mdl.plot_CM(y_test, y_pred_thresh)
    plt.savefig(f'{plotspath}MC{current_folder}_opt.png', bbox_inches='tight')
    plt.show()
    '''
    def plot_CM(y_test, y_pred):
        matrix = confusion_matrix(y_test, y_pred)
        names = ['True Neg','False Pos','False Neg','True Pos']
        counts = ['{0:0.0f}'.format(value) for value in
                        matrix.flatten()]
        percentages = ['{0:.2%}'.format(value) for value in
                             matrix.flatten()/np.sum(matrix)]
        labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in
                  zip(names,counts,percentages)]
        labels = np.asarray(labels).reshape(2,2)
        plot = sns.heatmap(matrix, annot=labels, fmt='', cmap='Blues')
        return plot'''
    # saving the predictions (0,1) in a csv file -- in probabilities 
    df = pd.DataFrame(y_pred, columns=['Predictions'])
    df['True'] = y_test
    df.to_csv(f"{predictionspath}predictions/%s.csv" % current_folder)
    print("Prediction saved.")


# Processing one RBP
- The chosen protein is '', which has high methylation rate and this could improve the ability of the model to discern. 

%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)

df_methylation = pd.read_csv(f'{resultspath}methylation_rate.csv')
df_high = df_methylation[df_methylation.methylation == 'high']
list_high_meth = list(df_high.RBP)

for folder in directory1:
    #if folder in list_high_meth:  # to be done for high methylation RBPs
    if folder == 'METAP2_K562':
        try:
            # preparation of the dataset
            path_figure = f'{plotspath}summary_metrics/{folder}/'
            x_train, y_train, x_val, y_val, x_test, y_test = mdl.prepare_original_dataset(path, folder) #prepares the dataset in one CV configuration

            y_labels = np.reshape(y_train, (len(y_train), ))
            class_weights = compute_class_weight(class_weight="balanced",
                                                 classes=np.unique(y_labels),
                                                 y=y_labels)
            class_weights = dict(zip(np.unique(y_train), class_weights))

            # STANDARD MODEL
            output_folder = '_standard_original'

            # fitting and testing 
            model = mdl.create_baseline_standard()
            mdl.training_model(model, class_weights, x_train, y_train, x_val, y_val, x_test, y_test, folder, resultspath, output_folder, path_figure)

            model_path = f'{resultspath}best_models/{output_folder}/%s' % folder  # path of the standard model, it has the same configuration for all RBPs 
            mdl.testing_model(model_path, folder, x_test, y_test, predictionspath, output_folder, path_figure)

            # TUNED MODEL
            output_folder = '_tuned_original'

            # prepare and save the best configuration per RBP to be used later
            mdl.prepare_best_model(x_train, y_train, x_val, y_val, x_test, y_test, resultspath, folder)  # this path is going to be used in all other tests. 

            # training the model on the original dataset
            model = keras.models.load_model(f'{resultspath}best_models/best_hps_model{folder}') #4 channels 
            mdl.training_model(model, class_weights, x_train, y_train, x_val, y_val, x_test, y_test, folder, resultspath, output_folder, path_figure) # best model is not trained on any data

            # evaluation and storage of the predicted labels
            model_path = f'{resultspath}best_models/{output_folder}/%s' % folder # trained model 
            mdl.testing_model(model_path, folder, x_test, y_test, predictionspath, output_folder, path_figure)
        except:  # I don't remember which type of error was raised 
            print(f'Exception occurred in {folder}')
            continue
            

In [ ]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

path = f'{datapath}processed/ENCODE-Sofia/'
directory1 = os.listdir(path)

df_methylation = pd.read_csv(f'{resultspath}methylation_rate.csv')
df_high = df_methylation[df_methylation.methylation == 'high']
list_high_meth = list(df_high.RBP)

for folder in directory1:
    #if folder in list_high_meth:  # to be done for high methylation RBPs
    if folder == 'PCBP1_HepG2':
        try:
            # preparation of the dataset
            path_figure = f'{plotspath}summary_metrics/{folder}/'
            x_train, y_train, x_val, y_val, x_test, y_test = mdl.prepare_original_dataset(path, folder) #prepares the dataset in one CV configuration
            #trying with small sized model
            x_train = x_train[:1200][:][:]
            y_train = y_train[:1200]
            x_val = x_val[:300][:][:]
            y_val = y_val[:300]
            
            print(len(x_train))
            print(len(x_val))
            print(len(y_train))
            print(len(y_val))
            
            y_labels = np.reshape(y_train, (len(y_train), ))
            class_weights = compute_class_weight(class_weight="balanced",
                                                 classes=np.unique(y_labels),
                                                 y=y_labels)
            class_weights = dict(zip(np.unique(y_train), class_weights))
        
            # STANDARD MODEL
            output_folder = '_standard_original'

            # fitting and testing 
            model = mdl.create_baseline_standard()
            mdl.training_model(model, class_weights, x_train, y_train, x_val, y_val, x_test, y_test, folder, resultspath, output_folder, path_figure)

            model_path = f'{resultspath}best_models/{output_folder}/%s' % folder  # path of the standard model, it has the same configuration for all RBPs 
            mdl.testing_model(model_path, folder, x_test, y_test, predictionspath, output_folder, path_figure)
            
            # ARTIFICIAL HYPERPARAMETERS
            output_folder = '_suggested_tuned_original'

            # fitting and testing 
            model = mdl.create_baseline_model((400, 4))
            mdl.training_model(model, class_weights, x_train, y_train, x_val, y_val, x_test, y_test, folder, resultspath, output_folder, path_figure)

            model_path = f'{resultspath}best_models/{output_folder}/%s' % folder  # path of the standard model, it has the same configuration for all RBPs 
            mdl.testing_model(model_path, folder, x_test, y_test, predictionspath, output_folder, path_figure)
        except:  # I don't remember which type of error was raised 
            print(f'Exception occurred in {folder}')
            continue